# Introduction to the ACCESS-NRI Intake catalog

**Aims**: This tutorial will introduce the ACCESS-NRI Intake catalog a show you how to use it to find and load model data for analysis

**Project membership requirements**:

 - If using the `xp65` conda environment: `xp65`, `dk92`, `fs38` and `p73`
 - If using the `hh5` conda environment: as above but also `hh5`

----

# Opening the catalog 

We'll start by opening the catalog and getting a feel for what it contains.

In [ ]:
import intake

catalog = intake.cat.access_nri

With that, we can now use `catalog` to search and load ~3 PB of data without having to know where the data is or how it's structured. 

The catalog includes a wide variety of climate data products. The "name" column gives the name of the data product and the other columns provide additional metadata associated with each product. As we'll demonstrate below, you can search on metadata in these columns to filter for data products that may be of interest to you. Scroll through the products below and get an idea for what each product is by looking at the entry in the description column.

In [ ]:
catalog

# Using the catalog

Each entry (row) in the catalog describes a data product comprising many datasets spread across many files (a "dataset" here is a set of files that can be readily opened and combined for analysis using xarray - more on this later). For example, in a given ACCESS-CM2 product, there may be a dataset of ocean variables at monthly frequency, atmospheric variables at monthly frequency etc. Each entry in the catalog has a corresponding Intake-ESM datastore that can be used to filter for datasets of interest based on metadata in the datastore and then to open those datasets using xarray.

The general process for using the catalog is as follows:

1. Search the ACCESS-NRI catalog for data products that are of interest to you.
2. Open the Intake-ESM datastore(s) for the product(s) of interest. 
3. Search the Intake-ESM datastore(s) for the datasets within each product that are of interest to you.
4. Open the datasets of interest as xarray Dataset(s).
5. Perform some analysis on the xarray Dataset(s).

This process is illustrated in the schematic below. Pink text indicates the methods used to perform each task.

<img src="./catalog_flow.svg" alt="Alternative text" />

If a user knows in advance the name of the product(s) they are after, then the first two steps above are not needed. Instead, those Intake-ESM datastores can be easily retreived directly from the catalog as demonstrated later on.

# Catalog filtering and data discovery

We can search on the columns in the ACCESS-NRI catalog. For example, we could search for all products that use the model `ACCESS-OM2`. The `search` method returns another catalog object with entries that satisfy our search criteria.

In [ ]:
catalog_filtered_example = catalog.search(model="ACCESS-OM2")
catalog_filtered_example

We can also combine queries in a search. For example, below we search for all products that use the model `ACCESS-OM2` and contain the variable `wdet100` at daily frequency.

In [ ]:
catalog.search(model="ACCESS-OM2", frequency="1day", variable="wdet100")

We can also use regex strings in our searches. For example, we could relax our query on variable to look for any variables starting with the letter `"w"`.

In [ ]:
catalog.search(model="ACCESS-OM2", frequency="1day", variable="w.*")

Note, metadata in the `realm` and `frequency` columns of the ACCESS-NRI catalog follow a standard vocabulary that is very similar to CMIP6 (but slightly more general):

 - `realm` may be one of:
   - `aerosol`,
   - `atmos`,
   - `atmosChem`,
   - `land`,
   - `landIce`,
   - `none`,
   - `ocean`,
   - `ocnBgchem`,
   - `seaIce`,
   - `unknown`
 - `frequency` may be one of (where `<int>` is an integer):
   - `fx`
   - `subhr`
   - `<int>hr`
   - `<int>day`
   - `<int>mon`
   - `<int>yr`
   - `<int>dec`
  
Some attempt has been made to use consistent model names in the `model` column (e.g. always use "ACCESS-OM2" for ACCESS-OM2), but model naming is not enforced. The variable names in the `variable` column are whatever they're called in the associated data product.

# Loading Intake sources

Remember that each entry in the catalog has an associated Intake-ESM datastore that keeps track of all the files in that product and how they fit together into datasets. There are three ways to open Intake-ESM datastores from the catalog, depening on your use case:

## 1. You know the name of the product you want

In this case, you can open the Intake-ESM datastore for that product directly as an attribute or key. For example

In [ ]:
datastore_example = catalog.by647

# Or

datastore_example = catalog["by647"]

## 2. You've filtered the catalog for the products you want and there are multiple remaining

In this case, you can open the Intake-ESM datastores for all entries in a catalog using the `to_source_dict` method. For example

In [ ]:
datastore_dict_example = catalog.search(model="ACCESS-OM2", frequency="1day", variable="wdet100").to_source_dict()

## 3. You've filtered the catalog for the products you want and there's only one remaining 

In this case, you can open the Intake-ESM datastore for the remaining product using the `to_source` method (note you could also use `to_source_dict` which would return a dictionary containing the Intake-ESM datastore, rather than the datastore itself). For example

In [ ]:
datastore_example = catalog.search(name="by647").to_source()

# Additional source metadata

Each Intake-ESM datastore has its own `.metadata` attribute that contains additional information about that experiment.

In [ ]:
catalog.by647.metadata

# An example workflow

(Note, up until this point we haven't actually opened any data from any products. However, below we access data from the `p73` project on Gadi. You will therefore need to be a member of this project to run this section.)

As a full example workflow, let's use the ACCESS-NRI catalog to carry out a simple analysis comparing anomalies of the global average ocean surface temperature from four ACCESS-CM2 data products. First we'll search directly for the products by their names and see what each of them are. 

In [ ]:
catalog_filtered = catalog.search(name=["bx944", "by473", "by647", "by578"])

catalog_filtered

Now we'll load the Intake-ESM datastores for those products using the `to_source_dict` method because there is more than one of them. This will take a few seconds.

In [ ]:
datastore_dict = catalog_filtered.to_source_dict()

datastore_dict

Now we can use our Intake-ESM datastores to open the data we want. You learned about Intake-ESM datastores earlier today. We'll show the basics here and refer to the Intake-ESM documentation [here](https://intake-esm.readthedocs.io/en/stable/index.html) for more information on how to use Intake-ESM datastores.

We're going to use dask to help us open and process the data from these datastores. We can start a local distributed dask cluster as follows.

In [ ]:
from distributed import Client

client = Client() #threads_per_worker=1)
client.dashboard_link

Note, it is very helpful to monitor the dask dashboard when working with dask. Click on the dask icon on the far left of the screen (three orange and red squares) and enter the text output by the previous cell in the search bar. Each of the different orange panels is a different dashboard that you can use to monitor what dask is doing. If you don't know which to choose, the "Task Stream", "Progress", "CPU" and "Workers Memory" diagnostics are a good start. Click on these, and drag the windows to where ever you want them in your JupyterLab.

In these ACCESS-CM2 products, monthly global average ocean surface temperatures are available in the variable `temp_surface_ave`. We can open an xarray Dataset with `temp_surface_ave` from the Intake-ESM datastore for each product as follows. The opening of these files will be parallelized across our dask cluster - watch you dask dashboard when you execute the following cell. Note that, because the datasets here are small, we've chosen to load them immediately using `load()`.

In [ ]:
dataset_dict = {
    name: datastore.search(variable="temp_surface_ave").to_dask().load()
    for name, datastore in datastore_dict.items()
}

The following function will calculate the anomalies of a provided dataset relative to the climatological mean of another dataset over the period 1971-2000. Here we'll calculate the anomalies for each product relative to the "bx944" standard historical run.

In [ ]:
def anomalize(dataset, dataset_baseline, period=slice("1971","2000")):
    """
    Calculate the anomalies of ds relative to the climatological mean of ds_baseline over period
    """
    clim = dataset_baseline.sel(time=period).mean("member").groupby("time.month").mean("time")
    return dataset.groupby("time.month") - clim

In [ ]:
anomalies_dict = {
    name: anomalize(dataset, dataset_dict["bx944"]) for name, dataset in dataset_dict.items()
}

To plot our data, we'll use a simple function that receives our dictionary of anomalies and plots the timeseries

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

def plot_temp_surface_ave(dataset_dict):
    for idx, (name, ds) in enumerate(dataset_dict.items()):
        data = ds["temp_surface_ave"].squeeze()
        data.plot.line(x="time", color=f"C{idx}", add_legend=False, label=name)
    
    plt.title("Global mean ocean surface temperature")
    plt.grid()
    plt.xlim([datetime(1969,1,1), datetime(2020,12,31)])
    
    # Add legend, removing duplicates due to multiple members
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

In [ ]:
plot_temp_surface_ave(anomalies_dict)

Maybe we'd also like to add some CMIP6 ACCESS-CM2 data to our plot? That's easy because the [NCI CMIP Intake-ESM datastores](https://opus.nci.org.au/pages/viewpage.action?pageId=213713098) are included in the ACCESS-NRI catalog.

In [ ]:
cmip6_datastore = catalog.search(name="cmip6.*", model="ACCESS-ESM1-5").to_source()

In this case however, there's no variable for global average ocean surface temperatures, so we'll have to calculate the area-weighted average ourselves from fields of ocean surface temperature.

In [ ]:
tos_dataset = cmip6_datastore.search(
    source_id="ACCESS-CM2", 
    table_id="Omon", 
    variable_id="tos", 
    experiment_id="historical", 
    member_id="r1i1p1f1",
    file_type="f"
).to_dask()

area_dataset = cmip6_datastore.search(
    source_id="ACCESS-CM2", 
    table_id="Ofx", 
    variable_id="areacello", 
    experiment_id="historical", 
    member_id="r1i1p1f1",
    file_type="f"
).to_dask()

temp_surface_ave = tos_dataset["tos"].weighted(area_dataset["areacello"]).mean(["i", "j"]).to_dataset(name="temp_surface_ave").load()

Again, we'll compute the anomalies relative to the "bx944" data for consistency and add this to our dictionary of anomalies to pass on to our plotting function.

In [ ]:
anomalies_dict["ACCESS-CM2 historical r1i1p1f1"] = anomalize(temp_surface_ave, dataset_dict["bx944"])

In [ ]:
plot_temp_surface_ave(anomalies_dict)

In [ ]:
client.close()